In [2]:
a = {'responseId': 'bddf13a3-02a3-4d21-b8ac-34ea95749b6b-415208ed', 
     'queryResult': {'queryText': '그림그릴래', 'parameters': {}, 
                     'allRequiredParamsPresent': True, 
                     'fulfillmentMessages': [{'text': {'text': ['']}}], 
                     'intent': {'name': 'projects/icksw-5d2f6/agent/intents/950875bb-8994-48b4-9baf-048fc02c62a4', 'displayName': 'Paint'}, 'intentDetectionConfidence': 1.0, 'languageCode': 'ko'}, 'originalDetectIntentRequest': {'payload': {}}, 'session': 'projects/icksw-5d2f6/agent/sessions/ab6e44d9-d0cf-8dc3-8ce0-9cd369519f47'}

In [5]:
a['queryResult']['intent']['displayName']

'Paint'